#Connect 4 Reinforcement Learning Model

##Setup

In [1]:
# Import
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
import random
from scipy.signal import convolve2d
import os

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
# Environment
RED = 1
YELLOW = -1
EMPTY = 0

class C4Env:
    def __init__(self, rows=6, cols=7): # 6 rows, 7 cols; stored from 5-0: 0 is top row
        self.rows = rows
        self.cols = cols
        self.shape = (rows, cols)
        self.board = np.zeros(self.shape, dtype=int)
        self.current_player = RED # player 1 and player -1

    def render(self, newline=True):
        display = '\n'.join([''.join(['R' if item==RED else 'Y' if item==YELLOW else '.' for item in row]) for row in self.board])
        if newline: display += '\n'
        print(display)

    def reset(self):
        self.board = np.zeros(self.shape, dtype=int)
        self.current_player = RED
        return self.board

    def legal_actions(self):
        return np.nonzero(self.board[0] == 0)[0] # which columns have empty top

    def step(self, action):
        assert action in self.legal_actions(), "Invalid action"
        # Find the first empty spot in the column
        for row in range(self.rows-1, -1, -1):
            if self.board[row, action] == EMPTY:
                self.board[row, action] = self.current_player
                break
        # Check for a win
        if self.winning_move(self.current_player):
            return self.board, self.current_player, True
        # Switch player
        self.current_player *= -1
        return self.board, None, False

    def winning_move(self, piece):
        # Define the kernel to use for convolution
        kernel = np.array([[1, 1, 1, 1]])

        # Check horizontal
        if (convolve2d(self.board == piece, kernel, mode='valid') == 4).any():
            return True

        # Check vertical
        if (convolve2d(self.board == piece, kernel.T, mode='valid') == 4).any():
            return True

        # Check positively sloped diagonals
        if (convolve2d(self.board == piece, np.eye(4), mode='valid') == 4).any():
            return True

        # Check negatively sloped diagonals
        if (convolve2d(self.board == piece, np.fliplr(np.eye(4)), mode='valid') == 4).any():
            return True

        return False


In [3]:
# Model

class QNet(nn.Module):
    def __init__(self, rows=6, cols=7):
        super(QNet, self).__init__()
        self.a = 64
        self.b = 64

        self.rows = rows
        self.cols = cols

        self.fc1 = nn.Linear(self.rows * self.cols, self.a)
        self.fc2 = nn.Linear(self.a, self.b)
        self.fc3 = nn.Linear(self.b, self.cols)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        return self.fc3(x)

# Agent

class QLearningAgent:
    def __init__(self, epsilon=0.5, ep_decay=0.9, gamma=0.9, learning_rate=0.1):
        self.p_network = QNet()
        self.t_network = QNet()
        self.optimizer = optim.Adam(self.p_network.parameters(), lr=learning_rate)
        self.epsilon = epsilon
        self.ep_decay = ep_decay
        self.gamma = gamma
        self.step = 0

    def get_action(self, state, valid_actions, boardEnv=None):
        if random.random() > self.epsilon:
            with torch.no_grad():
                state_tensor = torch.FloatTensor(state.flatten()).unsqueeze(0)
                q_values = self.p_network(state_tensor)
                valid_q_values = q_values[0][valid_actions]
                return valid_actions[torch.argmax(valid_q_values).item()]
        else:
            return random.choice(valid_actions)


    def update(self, state, action, reward, next_state, done):
        state_tensor = torch.FloatTensor(state.flatten()).unsqueeze(0)
        next_state_tensor = torch.FloatTensor(next_state.flatten()).unsqueeze(0)
        action_tensor = torch.LongTensor([action])
        reward_tensor = torch.FloatTensor([reward])

        q_values = self.p_network(state_tensor)
        next_q_values = self.t_network(next_state_tensor)

        q_value = q_values.gather(1, action_tensor.unsqueeze(1))
        next_q_value = next_q_values.max(1)[0].unsqueeze(1)
        expected_q_value = reward_tensor + (1 - done) * self.gamma * next_q_value

        loss = nn.MSELoss()(q_value, expected_q_value.detach())

        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()

    def update_tnet(self):
        self.t_network.load_state_dict(self.p_network.state_dict())

    def save(self, path):
        torch.save({
            'model_state_dict': self.p_network.state_dict(),
            'optimizer_state_dict': self.optimizer.state_dict(),
            'epsilon': self.epsilon,
            'gamma': self.gamma
        }, path)

    def load(self, path):
        if os.path.exists(path):
            checkpoint = torch.load(path)
            self.p_network.load_state_dict(checkpoint['model_state_dict'])
            self.optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
            self.epsilon = checkpoint['epsilon']
            self.gamma = checkpoint['gamma']
        else:
            print(f"No saved model found at {path}")

    def epsilon_decay(self):
        self.epsilon = max(self.ep_min, self.epsilon * self.ep_decay)


##Train

In [8]:
# Training loop
def train(num_episodes, epsilon=0.5, ep_decay=0.9, gamma=0.01, learning_rate=0.1):
    agent1 = QLearningAgent(epsilon=epsilon, ep_decay=ep_decay, gamma=gamma, learning_rate=learning_rate)
    agent2 = QLearningAgent(epsilon=epsilon, ep_decay=ep_decay, gamma=gamma, learning_rate=learning_rate)

    for episode in range(num_episodes):
        boardEnv = C4Env()
        board = boardEnv.board
        game_over = False
        turn = 0
        total_turns = 0

        while not game_over:
            # Get Action
            valid_actions = boardEnv.legal_actions()

            if turn == 0:
                action = agent1.get_action(board, valid_actions, boardEnv)
                piece = RED
            else:
                action = agent2.get_action(board, valid_actions, boardEnv)
                piece = YELLOW

            boardEnv.step(action)

            # Get Rewards
            reward = 0
            if boardEnv.winning_move(piece):
                reward = 1 if turn == 0 else -1
                game_over = True
            elif np.all(boardEnv.legal_actions == False):
                game_over = True

            next_state = board.copy()

            if turn == 0:
                agent1.update(board, action, reward, next_state, game_over)
            else:
                agent2.update(board, action, -reward, next_state, game_over)

            board = next_state
            turn = 1 - turn
            total_turns += 1

        if episode % 100 == 0:
            print(f"Episode {episode} completed: turns - {total_turns}")
            boardEnv.render()


    return agent1, agent2

# Train the agents
agent1, agent2 = train(1000)

Episode 0 completed: turns - 20
.......
.......
R....Y.
RY..YR.
RRRYYYY
YRYYRRR

Episode 100 completed: turns - 11
.......
.......
.......
..Y....
..R..RY
YYYRRRR

Episode 200 completed: turns - 14
.......
Y......
YR.....
YR.....
YY...R.
RRY.RYR

Episode 300 completed: turns - 34
R..R.YY
R..YYYY
Y..YRYY
RR.YRRR
RRYRYYR
RRYYRYR

Episode 400 completed: turns - 11
.......
.......
.....R.
..Y..R.
..Y..RR
.YY.YRR

Episode 500 completed: turns - 15
.......
...Y...
..RR...
..RYR..
..YYYR.
..RYRYR

Episode 600 completed: turns - 11
.......
.......
R......
RY.....
RY.Y...
RYYRR..

Episode 700 completed: turns - 23
....R..
....Y..
.Y.RR..
.RYRR..
YRYYYRY
YRYRRYR

Episode 800 completed: turns - 15
.......
.R.....
.R.....
.RR....
.RYR.YY
.YYRYYR

Episode 900 completed: turns - 11
.......
.......
...R...
...R...
Y..R..R
Y.YRYRY



##Evaluation

In [5]:
# Function to play a game using the trained model
def play_game(agent1, agent2):
    boardEnv = C4Env()
    board = boardEnv.board
    game_over = False
    turn = 0

    while not game_over:
        valid_actions = boardEnv.legal_actions()

        if turn == 0:
            action = agent1.get_action(board, valid_actions)
            piece = RED
        else:
            action = agent2.get_action(board, valid_actions)
            piece = YELLOW

        boardEnv.step(action)

        if boardEnv.winning_move(piece):
            boardEnv.render()
            print(f"Player {turn + 1} wins!")
            game_over = True
        elif np.all(boardEnv.legal_actions == False):
            boardEnv.render()
            print("It's a draw!")
            game_over = True

        boardEnv.render()
        turn = 1 - turn

# Play a game with the trained agents
play_game(agent1, agent2)

.......
.......
.......
.......
.......
..R....

.......
.......
.......
.......
.......
..R..Y.

.......
.......
.......
.......
..R....
..R..Y.

.......
.......
.......
.......
..R....
..R..YY

.......
.......
.......
.......
..R....
R.R..YY

.......
.......
.......
.......
Y.R....
R.R..YY

.......
.......
.......
.......
Y.R....
R.RR.YY

.......
.......
.......
Y......
Y.R....
R.RR.YY

.......
.......
.......
Y.R....
Y.R....
R.RR.YY

.......
.......
Y......
Y.R....
Y.R....
R.RR.YY

.......
.......
Y......
Y.R....
Y.R..R.
R.RR.YY

.......
.......
Y......
Y.R....
Y.R..R.
R.RRYYY

.......
.......
Y......
Y.R....
Y.R..RR
R.RRYYY

.......
Y......
Y......
Y.R....
Y.R..RR
R.RRYYY

Player 2 wins!
.......
Y......
Y......
Y.R....
Y.R..RR
R.RRYYY



##Player v Computer